# Extract, Transform & Load

Import libraries.

In [30]:
import pandas as pd
import sqlite3

Load CSV.

In [31]:
df_traffic_accidents = pd.read_csv("../data/raw/atus_anual_csv/conjunto_de_datos/atus_anual_2023.csv")

In [32]:
df_traffic_accidents.info

<bound method DataFrame.info of         COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  \
0       Municipal           1             1  2023    1        0          0   
1       Municipal           1             1  2023    1        0          0   
2       Municipal           1             1  2023    1        2         20   
3       Municipal           1             1  2023    1        3         20   
4       Municipal           1             1  2023    1        6          0   
...           ...         ...           ...   ...  ...      ...        ...   
396623  Municipal          32            56  2023   12       14         45   
396624  Municipal          32            56  2023   12       21         40   
396625  Municipal          32            56  2023   12       14         30   
396626  Municipal          32            56  2023   12        0          0   
396627  Municipal          32            56  2023   12       18          0   

        ID_DIA  DIASEMANA      

In [33]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'ANIO', 'MES', 'ID_HORA',
       'ID_MINUTO', 'ID_DIA', 'DIASEMANA', 'URBANA', 'SUBURBANA', 'TIPACCID',
       'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS', 'PASCAMION', 'OMNIBUS', 'TRANVIA',
       'CAMIONETA', 'CAMION', 'TRACTOR', 'FERROCARRI', 'MOTOCICLET',
       'BICICLETA', 'OTROVEHIC', 'CAUSAACCI', 'CAPAROD', 'SEXO', 'ALIENTO',
       'CINTURON', 'ID_EDAD', 'CONDMUERTO', 'CONDHERIDO', 'PASAMUERTO',
       'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO', 'CICLMUERTO', 'CICLHERIDO',
       'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO', 'NEHERIDO', 'CLASACC',
       'ESTATUS'],
      dtype='object')

In [34]:
df_traffic_accidents.dtypes

COBERTURA       object
ID_ENTIDAD       int64
ID_MUNICIPIO     int64
ANIO             int64
MES              int64
ID_HORA          int64
ID_MINUTO        int64
ID_DIA           int64
DIASEMANA       object
URBANA          object
SUBURBANA       object
TIPACCID        object
AUTOMOVIL        int64
CAMPASAJ         int64
MICROBUS         int64
PASCAMION        int64
OMNIBUS          int64
TRANVIA          int64
CAMIONETA        int64
CAMION           int64
TRACTOR          int64
FERROCARRI       int64
MOTOCICLET       int64
BICICLETA        int64
OTROVEHIC        int64
CAUSAACCI       object
CAPAROD         object
SEXO            object
ALIENTO         object
CINTURON        object
ID_EDAD          int64
CONDMUERTO       int64
CONDHERIDO       int64
PASAMUERTO       int64
PASAHERIDO       int64
PEATMUERTO       int64
PEATHERIDO       int64
CICLMUERTO       int64
CICLHERIDO       int64
OTROMUERTO       int64
OTROHERIDO       int64
NEMUERTO         int64
NEHERIDO         int64
CLASACC    

Create a date column.

In [35]:
df_traffic_accidents["fecha_hora"] = pd.to_datetime(dict(
        year=df_traffic_accidents["ANIO"],
        month=df_traffic_accidents["MES"],
        day=df_traffic_accidents["ID_DIA"],
        hour=df_traffic_accidents["ID_HORA"],
        minute=df_traffic_accidents["ID_MINUTO"]
    ), errors='coerce')

Drop NaT values.

In [36]:
df_traffic_accidents = df_traffic_accidents.dropna()
df_traffic_accidents["fecha_hora"].isna().sum()

np.int64(0)

Drop year, month, day, hour and minute columns.

In [37]:
df_traffic_accidents = df_traffic_accidents.drop(["ANIO","MES","ID_DIA","ID_HORA","ID_MINUTO"],axis=1)

In [38]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'DIASEMANA', 'URBANA',
       'SUBURBANA', 'TIPACCID', 'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS',
       'PASCAMION', 'OMNIBUS', 'TRANVIA', 'CAMIONETA', 'CAMION', 'TRACTOR',
       'FERROCARRI', 'MOTOCICLET', 'BICICLETA', 'OTROVEHIC', 'CAUSAACCI',
       'CAPAROD', 'SEXO', 'ALIENTO', 'CINTURON', 'ID_EDAD', 'CONDMUERTO',
       'CONDHERIDO', 'PASAMUERTO', 'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO',
       'CICLMUERTO', 'CICLHERIDO', 'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO',
       'NEHERIDO', 'CLASACC', 'ESTATUS', 'fecha_hora'],
      dtype='object')

Create a zone column.

In [39]:
zones = pd.DataFrame()
zones["URBANA"] = df_traffic_accidents["URBANA"].replace("Sin accidente en esta zona", pd.NA)
zones["SUBURBANA"] = df_traffic_accidents["SUBURBANA"].replace("Sin accidente en esta zona", pd.NA)

df_traffic_accidents["zona"] = zones["URBANA"].combine_first(zones["SUBURBANA"])

Fill rows with undefined zone.

In [40]:
df_traffic_accidents["zona"] = df_traffic_accidents["zona"].fillna('No especificado')

Drop URBANA and SUBURBANA columns.

In [41]:
df_traffic_accidents = df_traffic_accidents.drop(["URBANA","SUBURBANA"],axis=1)

In [42]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'DIASEMANA', 'TIPACCID',
       'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS', 'PASCAMION', 'OMNIBUS', 'TRANVIA',
       'CAMIONETA', 'CAMION', 'TRACTOR', 'FERROCARRI', 'MOTOCICLET',
       'BICICLETA', 'OTROVEHIC', 'CAUSAACCI', 'CAPAROD', 'SEXO', 'ALIENTO',
       'CINTURON', 'ID_EDAD', 'CONDMUERTO', 'CONDHERIDO', 'PASAMUERTO',
       'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO', 'CICLMUERTO', 'CICLHERIDO',
       'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO', 'NEHERIDO', 'CLASACC',
       'ESTATUS', 'fecha_hora', 'zona'],
      dtype='object')

Change columns name.

In [43]:
rename_columns = {
    "ID_ENTIDAD": "entidad_id",
    "ID_MUNICIPIO": "municipio_id",
    "TIPACCID": "tipo_accidente",
    "AUTOMOVIL": "automovil",
    "CAMPASAJ": "camioneta_pasajeros",
    "MICROBUS": "microbus",
    "PASCAMION": "camion_urbano",
    "OMNIBUS": "autobus",
    "TRANVIA": "tranvia",
    "CAMIONETA": "camioneta_carga",
    "CAMION": "camion_carga",
    "TRACTOR": "tractor",
    "FERROCARRI": "ferrocarril",
    "MOTOCICLET": "motocicleta",
    "BICICLETA": "bicicleta",
    "OTROVEHIC": "otro_vehiculo",
    "CAUSAACCI": "causa_accidente",
    "CAPAROD": "superficie_rodamiento",
    "SEXO": "sexo",
    "ALIENTO": "aliento_alcoholico",
    "CINTURON": "cinturon",
    "ID_EDAD": "edad",
    "CONDMUERTO":"conductor_muerto",
    "CONDHERIDO": "conductor_herido",
    "PASAMUERTO": "pasajero_muerto",
    "PASAHERIDO": "pasajero_herido",
    "PEATMUERTO": "peaton_muerto",
    "PEATHERIDO": "peaton_herido",
    "CICLMUERTO": "ciclista_muerto",
    "CICLHERIDO": "ciclista_herido",
    "OTROMUERTO": "otro_muerto",
    "OTROHERIDO": "otro_herido",
    "NEMUERTO": "no_espec_muerto",
    "NEHERIDO": "no_espec_herido"
}

for original, new in rename_columns.items():
    if original in df_traffic_accidents.columns:
        df_traffic_accidents.rename(columns={original: new}, inplace=True)

In [44]:
df_traffic_accidents.columns

Index(['COBERTURA', 'entidad_id', 'municipio_id', 'DIASEMANA',
       'tipo_accidente', 'automovil', 'camioneta_pasajeros', 'microbus',
       'camion_urbano', 'autobus', 'tranvia', 'camioneta_carga',
       'camion_carga', 'tractor', 'ferrocarril', 'motocicleta', 'bicicleta',
       'otro_vehiculo', 'causa_accidente', 'superficie_rodamiento', 'sexo',
       'aliento_alcoholico', 'cinturon', 'edad', 'conductor_muerto',
       'conductor_herido', 'pasajero_muerto', 'pasajero_herido',
       'peaton_muerto', 'peaton_herido', 'ciclista_muerto', 'ciclista_herido',
       'otro_muerto', 'otro_herido', 'no_espec_muerto', 'no_espec_herido',
       'CLASACC', 'ESTATUS', 'fecha_hora', 'zona'],
      dtype='object')

Drop unnecessary columns.

In [45]:
df_traffic_accidents = df_traffic_accidents.drop(['COBERTURA', 'DIASEMANA', 'CLASACC', 'ESTATUS'], axis=1)
df_traffic_accidents.columns

Index(['entidad_id', 'municipio_id', 'tipo_accidente', 'automovil',
       'camioneta_pasajeros', 'microbus', 'camion_urbano', 'autobus',
       'tranvia', 'camioneta_carga', 'camion_carga', 'tractor', 'ferrocarril',
       'motocicleta', 'bicicleta', 'otro_vehiculo', 'causa_accidente',
       'superficie_rodamiento', 'sexo', 'aliento_alcoholico', 'cinturon',
       'edad', 'conductor_muerto', 'conductor_herido', 'pasajero_muerto',
       'pasajero_herido', 'peaton_muerto', 'peaton_herido', 'ciclista_muerto',
       'ciclista_herido', 'otro_muerto', 'otro_herido', 'no_espec_muerto',
       'no_espec_herido', 'fecha_hora', 'zona'],
      dtype='object')

Create DB.

In [46]:
conn = sqlite3.connect("../data/DB/accidentes_trafico_mexico")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

Create tables.

In [47]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS vehiculos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    automovil INTEGER,
    camioneta_pasajeros INTEGER,
    microbus INTEGER,
    camion_urbano INTEGER,
    autobus INTEGER,
    tranvia INTEGER,
    camioneta_carga INTEGER,
    camion_carga INTEGER,
    tractor INTEGER,
    ferrocarril INTEGER,
    motocicleta INTEGER,
    bicicleta INTEGER,
    otro_vehiculo INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS heridos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    conductor_herido INTEGER
    pasajero_herido INTEGER,
    peaton_herido INTEGER,
    ciclista_herido INTEGER,
    otro_herido INTEGER,
    no_espec_herido INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS muertos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    conductor_muerto INTEGER,
    pasajero_muerto INTEGER,
    peaton_muerto INTEGER,
    ciclista_muerto INTEGER,
    otro_muerto INTEGER,
    no_espec_muerto INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS entidades (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS municipios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre INTEGER,
    entidad_id INTEGER,
    FOREIGN KEY (entidad_id) REFERENCES entidades(id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS accidentes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    municipio_id INTEGER,
    tipo_accidente TEXT,
    causa_accidente TEXT,
    superficie_rodamiento TEXT,
    sexo TEXT,
    aliento_alcoholico TEXT,
    cinturon TEXT,
    edad INTEGER,
    fecha_hora TEXT,
    zona TEXT,
    vehiculos_id INTEGER,
    heridos_id INTEGER,
    muertos_id INTEGER,
    FOREIGN KEY (vehiculos_id) REFERENCES vehiculos(id),
    FOREIGN KEY (heridos_id) REFERENCES heridos(id),
    FOREIGN KEY (muertos_id) REFERENCES muertos(id),
    FOREIGN KEY (municipio_id) REFERENCES municipios(id)
);
""")

Split df_traffic_accidents.

In [48]:
df_vehiculos = df_traffic_accidents[['automovil','camioneta_pasajeros', 'microbus', 'camion_urbano', 'autobus','tranvia', 'camioneta_carga', 
                                     'camion_carga', 'tractor', 'ferrocarril', 'motocicleta', 'bicicleta', 'otro_vehiculo']]
df_heridos = df_traffic_accidents[['conductor_herido', 'pasajero_herido', 'peaton_herido', 'ciclista_herido', 'otro_herido', 'no_espec_herido']]
df_muertos = df_traffic_accidents[['conductor_muerto', 'pasajero_muerto', 'peaton_muerto', 'ciclista_muerto', 'otro_muerto', 'no_espec_muerto']]
df_accidentes = df_traffic_accidents[['entidad_id', 'municipio_id', 'tipo_accidente', 'causa_accidente', 'superficie_rodamiento', 'sexo', 
                                      'aliento_alcoholico', 'cinturon', 'edad', 'fecha_hora', 'zona']]

Add id columns to the dataframes.

In [49]:
df_vehiculos.insert(0, "id", range(1, len(df_vehiculos) + 1))
df_heridos.insert(0, "id", range(1, len(df_heridos) + 1))
df_muertos.insert(0, "id", range(1, len(df_muertos) + 1))
df_accidentes.insert(0, "id", range(1, len(df_accidentes) + 1))

Load entidades and minicipios datasets.

In [50]:
df_entidades = pd.read_csv("../data/raw/atus_anual_csv/catalogos/tc_entidad.csv")
df_municipios = pd.read_csv("../data/raw/atus_anual_csv/catalogos/tc_municipio.csv")

rename_entidades = {
    "ID_ENTIDAD": "id",
    "NOM_ENTIDAD": "nombre"
}

for original, new in rename_entidades.items():
    if original in df_entidades.columns:
        df_entidades.rename(columns={original: new}, inplace=True)

rename_municipios = {
    "ID_ENTIDAD": "entidad_id",
    "ID_MUNICIPIO": "municipio_id",
    "NOM_MUNICIPIO": "nombre"
}

for original, new in rename_municipios.items():
    if original in df_municipios.columns:
        df_municipios.rename(columns={original: new}, inplace=True)

Add id column to municipios.

In [51]:
df_municipios.insert(0, "id", range(1, len(df_municipios) + 1))

Replace the entidad_id and municipio_id with the new id from df_municipios in df_accidentes.

In [52]:
df_accidentes = df_accidentes.merge(df_municipios[['id', 'entidad_id', 'municipio_id']],on=['entidad_id', 'municipio_id'],how='left')

Drop entidad_id and municipio_id in df_accidentes.

In [53]:
df_accidentes = df_accidentes.drop(['entidad_id', 'municipio_id'], axis=1)
df_accidentes.columns

Index(['id_x', 'tipo_accidente', 'causa_accidente', 'superficie_rodamiento',
       'sexo', 'aliento_alcoholico', 'cinturon', 'edad', 'fecha_hora', 'zona',
       'id_y'],
      dtype='object')

Drop municipio_id in df_municipios.

In [54]:
df_municipios = df_municipios.drop(['municipio_id'], axis=1)
df_municipios.columns

Index(['id', 'entidad_id', 'nombre'], dtype='object')

Rename id´s columns in df_accidentes.

In [55]:
rename_id ={
    "id_x": "id",
    "id_y": "municipio_id"
}

for original, new in rename_id.items():
    if original in df_accidentes.columns:
        df_accidentes.rename(columns={original: new}, inplace=True)

Load DataFrame to te DB.

In [56]:
df_entidades.to_sql("entidades",conn,if_exists="replace",index=False)
df_municipios.to_sql("municipios",conn,if_exists="replace",index=False)
df_vehiculos.to_sql("vehiculos",conn,if_exists="replace",index=False)
df_muertos.to_sql("muertos",conn,if_exists="replace",index=False)
df_heridos.to_sql("heridos",conn,if_exists="replace",index=False)
df_accidentes.to_sql("accidentes",conn,if_exists="replace",index=False)

396623